In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
data = pd.read_csv('bikesharing_clean.csv')
data.head()

,season,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,1,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,985
1,1,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,801
2,1,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,1349
3,1,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,1562
4,1,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,1600


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      731 non-null    int64  
 1   mnth        731 non-null    int64  
 2   holiday     731 non-null    int64  
 3   weekday     731 non-null    int64  
 4   workingday  731 non-null    int64  
 5   weathersit  731 non-null    int64  
 6   temp        731 non-null    float64
 7   atemp       731 non-null    float64
 8   hum         731 non-null    float64
 9   windspeed   731 non-null    float64
 10  cnt         731 non-null    int64  
dtypes: float64(4), int64(7)
memory usage: 62.9 KB


In [13]:
X = data.drop('cnt', axis=1)
y = data['cnt']

## Forward selection
We will select a model for each number of features using $R^2$ and then select the best of all $p+1$ models using AIC or Adjusted-$R^2$. Here, $p$ is the number of features.

First let's write down the functions for Adjusted-$R^2$ and AIC.

In [76]:
def adjustedR2(r2, n, p):
    return 1-((1-r2**2)*(n-1))/(n-p-1)

In [77]:
def aic(y_test, y_pred, p):
    ss_res = sum((y_test-y_pred)**2)
    n = y_test.shape[0]
    return n*np.log(ss_res/n)+2*p

In [78]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [79]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [119]:
all_cols = list(X.columns.copy())
selected_cols = []
rem_cols = all_cols.copy()

r2_max = r2_score(y_test, np.ones(len(y_test))*y_test.mean())

best_adjr2_idx = -1
best_adjr2 = adjustedR2(r2_max, y_test.shape[0], 0)

best_aic_idx = -1
best_aic = aic(y_test, np.ones(len(y_test))*y_test.mean(), 0)

for i in range(len(all_cols)):
    idx_max = 0
    for j in range(len(all_cols)-i):
        lr = LinearRegression(fit_intercept=True)
        lr.fit(np.c_[X_train[selected_cols], X_train[rem_cols[j]]], y_train)
        y_pred = lr.predict(np.c_[X_test[selected_cols], X_test[rem_cols[j]]])
        temp_r2 = r2_score(y_test, y_pred)
        if temp_r2 > r2_max:
            r2_max = temp_r2
            idx_max = j
    selected_cols.append(rem_cols.pop(idx_max))

    lr = LinearRegression(fit_intercept=True)
    lr.fit(X_train[selected_cols],y_train)
    y_pred = lr.predict(X_test[selected_cols])

    if (temp_adj:=adjustedR2(r2_score(y_test, y_pred), y_test.shape[0], i + 1)) > best_adjr2:
        best_adjr2 = temp_adj
        best_adjr2_idx = i

    if (temp_aic:=aic(y_test, y_pred, i + 1)) < best_aic:
        best_aic = temp_aic
        best_aic_idx = i    

print(f'Best AIC is: {best_aic}')
if best_aic_idx == -1:
    print('Best model is the null model')
else:
    print(f'Best subset as per AIC is: {all_cols[:best_aic_idx+1]}')
print()

print(f'Best adj.R2 is: {best_adjr2}')
if best_adjr2_idx == -1:
    print('Best model is the null model')
else:
    print(f'Best subset as per adj.R2 is: {all_cols[:best_adjr2_idx+1]}')

Best AIC is: 2145.8420646244867
Best subset as per AIC is: ['season', 'mnth', 'holiday']

Best adj.R2 is: 0.2415847698918906
Best subset as per adj.R2 is: ['season', 'mnth', 'holiday', 'weekday']


## Backward Selection

In [152]:
all_cols = list(X.columns.copy())
selected_cols = []
rem_cols = all_cols.copy()

#fit the full model we start with
lr = LinearRegression(fit_intercept=True)

lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
r2_max = r2_score(y_test, y_pred)

best_adjr2_idx = -1
best_adjr2 = adjustedR2(r2_max, y_test.shape[0], len(all_cols))

best_aic_idx = -1
best_aic = aic(y_test, y_pred, len(all_cols))

best_aic_selected = None
best_adjr2_selected = None

for i in range(len(all_cols)):
    idx_max = 0

    best_candidate_r2 = -np.inf
    best_candidate_pred = None

    for j in range(len(rem_cols)):
        if len(rem_cols) > 1:
            cols_after = rem_cols.copy()
            cols_after.pop(j)
            lr = LinearRegression(fit_intercept=True)
            lr.fit(X_train[cols_after], y_train)
            y_pred_candidate = lr.predict(X_test[cols_after])
        else:
            y_pred_candidate = np.ones(len(y_test)) * y_train.mean()

        temp_r2 = r2_score(y_test, y_pred_candidate)

        if temp_r2 > best_candidate_r2:
            best_candidate_r2 = temp_r2
            idx_max = j
            best_candidate_pred = y_pred_candidate

    removed = rem_cols.pop(idx_max)
    selected_cols.append(removed)

    if len(rem_cols) > 0:
        y_pred = best_candidate_pred
    else:
        y_pred = np.ones(len(y_test)) * y_train.mean()

    r2_max = r2_score(y_test, y_pred)

    p_after = len(rem_cols)

    temp_adj = adjustedR2(r2_score(y_test, y_pred), y_test.shape[0], p_after)
    temp_aic = aic(y_test, y_pred, p_after)

    if temp_adj > best_adjr2:
        best_adjr2 = temp_adj
        best_adjr2_idx = i
        best_adjr2_selected = rem_cols.copy()  

    if temp_aic < best_aic:
        best_aic = temp_aic
        best_aic_idx = i
        best_aic_selected = rem_cols.copy()  

print(f'Best AIC is: {best_aic}')
if best_aic_idx == -1:
    print('Best model is the null model')
else:
    print(f'Best subset as per AIC is: {best_aic_selected}')
print()

print(f'Best adj.R2 is: {best_adjr2}')
if best_adjr2_idx == -1:
    print('Best model is the null model')
else:
    print(f'Best subset as per adj.R2 is: {best_adjr2_selected}')


Best AIC is: 2145.8420646244867
Best subset as per AIC is: ['season', 'weathersit', 'atemp']

Best adj.R2 is: 0.24158476989188793
Best subset as per adj.R2 is: ['season', 'weekday', 'weathersit', 'atemp']
